ANN-PSO for Cubic function approximation using sigmoid activation function: ANN architecture ANN Forward Pass Paricle Swarm Data Execution

In [129]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [130]:
#ANN architecture:
#Number of neurons in each layer:
Input = 1
Hidden = 15
Output = 1
#Number of hidden layers:
Layers = 45

In [131]:
#ANN information pass
def forward_pass(input_X, output_Y, W):

     
    if isinstance(W, Particle):
        W = W.p

    for i in range(Layers):
    #weight and bias from input to hidden layers     
        w1 = W[0 : Input * Hidden].reshape((Input, Hidden))
        b1 = W[Input * Hidden:(Input * Hidden) + Hidden].reshape((Hidden, ))
    
    
   
    #weight and bias from hidden to output
    w4 = W[(Input * Hidden) + Hidden:(Input * Hidden) + Hidden +\
        (Hidden * Output)].reshape((Hidden, Output))
    b4 = W[(Input * Hidden) + Hidden + (Hidden * Output): (Input *\
        Hidden) + Hidden + (Hidden * Output) + Output].reshape((Output, ))

     #infomration pass using sigmoid activation function
    z1 = np.dot(input_X, w1) + b1
    a1 = tanh(z1)
    
       
    if Layers > 1:
          for i in range(Layers):
                z1 = np.dot(a1, w1.T) + b1
                a1 = tanh(z1)
                
      
            
    z2 = np.dot(a1, w4) + b4
    output_from_outputlayer = sigmoid(z2)
    
   
    #mean squared error   
    mserror1 = (np.power((output_from_outputlayer - output_Y), 2)) # **2
    mserror = (mserror1.sum()/100)
   
    print(mserror)
                    
    return mserror

In [132]:
class Particle:
   
    def __init__(self, dims, p_range, v_range):
        
        #dims = number of dimensions
        #p= position of the particle in the dimentional space
        #v = velocity of the particle in the dimensional space
        #p_range = range of dimensions 
        #v_range = range of velocities
        
        #let the position and velocity of a praticle be randomly selected from uniform distribution
        self.p = np.random.uniform(p_range[0], p_range[1], (dims, ))
        self.v = np.random.uniform(v_range[0], v_range[1], (dims, )) 
        
        #the best position for any particle can be positive infinity
        self.pbest = float('inf') #-1 #np.inf
        
        #define particles best position based on specified dimensions
        self.pbestpos = np.zeros((dims, ))

In [133]:
#collect MSE for plotting errors
MSE = []

#define swarm and oprtimization function 
class Swarm:
    
    def __init__(self, particle_no, dims, p_range, v_range, alpha_range, component):
        
        #particle_no = number of particles
        #dims = dimensions
        #p_range = range of dimensions 
        #v_range = range of velocities
        #alpha_range = inertia weight range
        #components = cognetive and social components
        
        
        #define the swarm with praticles from Class Particle
        self.p = np.array([Particle(dims, p_range, v_range) for i in range(particle_no)])
        
        #define global best as infinity
        self.gbest =float('inf')
        
        #define global best poition
        self.gbestpos = np.zeros((dims, ))
        
        self.p_range = p_range
        self.v_range = v_range
        
        self.alpha_range = alpha_range
        
        #components
        self.component0 = component[0] #cognetive
        self.component1 = component[1] #social
        self.dims = dims
        
        
     # optimization function    
    def optimize(self, function, input_X, output_Y,  print_step,  iter):
        informants = 6 
        for i in range(iter):
            for particle in self.p:
                fitness = function(input_X, output_Y, particle.p)

                #select the particles best position
                if  fitness < particle.pbest:
                    particle.pbest = fitness
                    particle.pbestpos = particle.p.copy()
                    
            for i in range(informants):
                for particle in self.p:          
                
                #select the global best position    
                 if fitness< self.gbest:
                    self.gbest = fitness
                    self.gbestpos = particle.p.copy()
                    

            for particle in self.p:
               
                alpha = np.random.uniform(self.alpha_range[0], self.alpha_range[1], 1)[0]
                
                #velocity update formula
                particle.v = alpha * particle.v + (self.component0 * np.random.uniform(0.0, 1.0, (self.dims, )) * \
                (particle.pbestpos - particle.p)) + (self.component1 * np.random.uniform(0.0, 1.0, (self.dims, )) \
                * (self.gbestpos - particle.p))
                
                #factor to enable relative velocity update
                epsilon = 0.2
               
                # particle position update  
                particle.p = particle.p + epsilon * particle.v
                
                
              #collect MSE for plotting
            if i % print_step == 0:
                 MSE.append(fitness)
        print("global best (lowest) error: ", self.gbest)

In [134]:
#activation function and data conversion function
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [135]:
#activation function and data conversion function
def tanh(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

In [136]:
#X input for which function approximation is carried out
import csv
with open("X_linear.csv", 'r') as f:
    Xin = list(csv.reader(f, delimiter=";"))
import numpy as np
Xin = np.array(Xin[1:], dtype=np.float)
input_X= Xin

In [137]:
#for cubic conversion for cubic approximation
def cubic(x):
    return x**3

In [138]:
output_Y = cubic(input_X)

In [139]:
    #PSO parameters
    
    #number of particles
    no_particles = 10
    
    #dimensions
    dims = (Input * Hidden) + Hidden + (Hidden * Output) + Output
    
    #weight range
    w_range = (-0.1,0.1)
    
    #learning rate range
    l_range = (0.15, 0.25)
    
    #weight velocity range
    alpha_range = (0.9,0.9)  
    
    #cognetive and social compenents
    components = (1,3)  
    
    #create a swarm object
    s = Swarm(no_particles, dims, w_range, l_range, alpha_range, components)
    
    #define number of iterations
    iterations = 100
    
    #call the optimization function on swarm object
    s.optimize(forward_pass, input_X, output_Y, 5,iterations)
    
    

0.38037413500609574
0.38539176326322727
0.37911648858034086
0.3801036836877916
0.4155032921918022
0.40057803102017964
0.3855818126305218
0.4272450809274947
0.37867445501581065
0.4064300052559255
0.40727094276490866
0.4019725149650823
0.40216934305641927
0.41323588756926377
0.42737896181163704
0.40959423057066324
0.3920704501518362
0.4439866263153827
0.4049602069604025
0.4192306560023613
0.48482927807625936
0.4936906588054977
0.48630122678849586
0.5052524817991244
0.5216606187104014
0.4569821641125448
0.4316402939657514
0.47304867172219184
0.49802222083017794
0.4610603876767146
0.4506585843489576
0.45900390366935295
0.45277175918662943
0.4347953601398201
0.5260302165352853
0.4470566461002627
0.43671411617074407
0.4374669107361088
0.48474167828980286
0.41009108230113867
0.38262824814470475
0.3710675286786024
0.38898687685900873
0.3891566869123919
0.4083565814317884
0.38644485562279746
0.39402157741152066
0.36344978223688307
0.3923903140455421
0.3756667323079062
0.37510581129537124
0.3603

0.37469318474922153
0.36768406625770905
0.37269716097968614
0.3719423077823577
0.3692137605672532
0.3505057142960199
0.3659516280002099
0.3719599245932735
0.37160699487730653
0.3694882644399804
0.37327044003937937
0.3675885296592518
0.37086736983818885
0.370384768151965
0.3666932752366019
0.3455647599276876
0.3695270084757165
0.371967094885809
0.3715076910103726
0.37296039708787904
0.37245274258377753
0.36805991895244744
0.3709816604607491
0.3700396204835193
0.36767200404482364
0.3523936892081971
0.3741153690121055
0.3734120123742779
0.3725404959925027
0.37488882068769663
0.3705842202189678
0.36981239889789647
0.37149190108420305
0.37180453361189814
0.3709966105144025
0.3647625610036698
0.37692774209400814
0.3746359408596948
0.37386692976152075
0.37584733350097826
0.3703471695746701
0.3719729205150042
0.36883564121216006
0.3730970912368761
0.3763108558474583
0.37718727584377676
0.3774169803590847
0.37633829233857585
0.3759049695024478
0.374907127936425
0.36957773507998903
0.37407253761

0.3758622709216974
0.37277015633451915
0.37460637041416567
0.3719181868971468
0.37171433138982124
0.3714269618636578
0.37445840216124326
0.36981839522932086
0.3490107991121464
0.3687614077529426
0.3751487253865733
0.3725377732767564
0.373713241944546
0.3710072109946473
0.37019436374112535
0.3655282772788489
0.3744241171388903
0.3705782998853497
0.33530607984896377
0.3704141644356123
0.3732316964235424
0.37327270420422026
0.3724857049648725
0.3708579503012704
0.36782160651494505
0.3608845740173329
0.3742262249803165
0.3723321017489088
0.32434833689145653
0.37297838421287
0.37170988514186276
0.3739059712833236
0.37166908872321924
0.36980059463435405
0.36601992320419674
0.36198096461755724
0.3735653001057392
0.375828693972277
0.3324726891195703
0.3736337117786395
0.3708596745441019
0.37521597156066583
0.37115260033486536
0.3707539695871683
0.3664374036161406
0.3656667439210033
0.373122440543926
0.3759075255239005
0.3420895397295688
0.37280443554610926
0.36998416898986863
0.376447119005197

<ipython-input-140-8eaaf908276e>:6: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  g = graph_ms.add_subplot()


In [140]:
%matplotlib inline
import matplotlib.pyplot as plt
graph_ms = plt.figure()
g =plt.xlabel("Number of Iterations")
g =plt.ylabel("Mean Squared Error")
g = graph_ms.add_subplot()
g.plot(MSE)